In [11]:
from typing import List, Optional
from enum import Enum
from pydantic import BaseModel, Field, FilePath

In [12]:
class vdW1(BaseModel):
    a: float = Field(annotation="The parameter a in vdW")
    b: float = Field(annotation="The parameter b in vdW")
        
class vdW(BaseModel):
    TcK: List[float] = Field(alias='Tcrit / K', annotation='The critical temperaturess in K')
    pcK: List[float] = Field(alias='pcrit / Pa', annotation='The critical pressures in Pa')
        
# Helper types for cubic models
class AlphaOptions(str, Enum):
    Twu = "Twu"
class CubicAlpha(BaseModel):
    type_: AlphaOptions = Field(alias='type', annotation='The type of the alpha function')
    c: List[float] = Field(min_length=3, max_length=3, annotation='The set of coefficients')
        
class PR(BaseModel):
    TcK: List[float] = Field(alias='Tcrit / K', annotation='The critical temperaturess in K')
    pcK: List[float] = Field(alias='pcrit / Pa', annotation='The critical pressures in Pa')
    acentric: List[float] = Field(annotation='The acentric factors')
    alpha: CubicAlpha = Field(default=None, annotation='The alpha function used in the attractive part')
    kmat: Optional[List[List[float]]] = Field(default=None, annotation="The NxN matrix of kij factors")
        
SRK = PR

class multifluid(BaseModel):
    components: List[FilePath | str]
    root: Optional[str]
    departure: FilePath | List[dict]
    BIP: FilePath | List[dict]
    flags: Optional[object]

In [14]:
import json
from pydantic.json_schema import GenerateJsonSchema
    
examples = [
    {"kind": "vdW1", "model":{"a": 1.0, "b": 2.0}},
    {"kind": "PR", "model": {"Tcrit / K": [190], "pcrit / Pa": [3.5e6], "acentric": [0.11], "alpha": {"type": "Twu", "c":[1,2,3]}}}
]
for example in examples:
    klass = locals()[example['kind']]
    klass.model_validate(example['model'])
    
schemas = {}
for klass in [vdW1, vdW, PR, SRK]:
    schemas[klass.__name__] = klass.model_json_schema()
with open("schemas.json",'w') as fp:
    fp.write(json.dumps(schemas, indent=2))